## CNN with GLOVE Embeddings for Sentence Classification

In [1]:
# Imports
!pip install portalocker
!pip install torchmetrics

import argparse
import logging
import time
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import DATASETS
from torchtext.prototype.transforms import load_sp_model, PRETRAINED_SP_MODEL, SentencePieceTokenizer
from torchtext.utils import download_from_url
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torchtext.vocab import GloVe
from tqdm import tqdm

torch.autograd.set_detect_anomaly(True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 4.0 MB/s eta 0:00:00


In [2]:
# Initialize some variables
DATASET = "AG_NEWS"
DATA_DIR = ".data"
DEVICE = "cpu"
EMBED_DIM = 300
LR = 1.0
BATCH_SIZE = 128
NUM_EPOCHS = 5
PADDING_VALUE = 0
PADDING_IDX = PADDING_VALUE

### Prepare the Vocabulary/Tokenize Text

In [6]:
# Get basic tokenizer
basic_english_tokenizer =get_tokenizer("basic_english")
TOKENIZER = basic_english_tokenizer

# This takes a (x, y) pair from data_iter and returns tokenized list of words
# For every sentence, it should yield a tokenized set of string
# "a b c d" -> ["a", "b", "c", "d"]
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield TOKENIZER(text)

# For text, return the tokens for each word
# This might work like "The man walks" -> ["the", "man", "walks"] -> [17, 123, 5]
def text_pipeline(text):
    return VOCAB(TOKENIZER(text))

# Return the label as an integer between 0 and num_classes - 1 inclusive
def label_pipeline(label):
    return int(label) - 1

# For each batch, return a tensor of labels and a tensor of ints representing the words in associated sentences
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        # Get the label from {1, 2, 3, 4} to {0, 1, 2, 3}
        label_list.append(label_pipeline(_label))

        # Return a list of ints
        # This basically returns a tensor of the token ids and then puts them into text_list
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        # Append to text_list
        text_list.append(processed_text.clone().detach())


    # Pad and make into tensors as needed
    label_list = torch.tensor(label_list, dtype=torch.int64)

    text_list = pad_sequence(text_list, batch_first=True)

    return label_list.to(DEVICE), text_list.to(DEVICE)

# Load data
train_iter, test_iter = DATASETS[DATASET]()
num_class = len(set([label for (label, _) in train_iter]))
print(f"The number of classes is {num_class} ...")
VOCAB = build_vocab_from_iterator(yield_tokens(train_iter), specials=('<pad>', '<unk>'))
# Make the default index the same as that of the unk_token.
VOCAB.set_default_index(VOCAB['<unk>'])
print("Total number of words:",len(VOCAB))

# Define test/train sets
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# Set up the dataloader
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

# Peek at the training data
for yb, xb in train_dataloader:
    print("Shape y:",yb.shape, "Shape x:",xb.shape)
    print( "x:", xb)
    break

The number of classes is 4 ...
Total number of words: 95812
Shape y: torch.Size([128]) Shape x: torch.Size([128, 114])
x: tensor([[ 9742,   896,    12,  ...,     0,     0,     0],
        [ 2143, 39793,     5,  ...,     0,     0,     0],
        [ 4078,  3609,  9705,  ...,     0,     0,     0],
        ...,
        [ 3672, 39012,   147,  ...,     0,     0,     0],
        [ 5579,   721,  8684,  ...,     0,     0,     0],
        [ 1096,     5,   663,  ...,     0,     0,     0]])


### Get GLOVE embeddings

In [8]:
# GLOVE is a pretrained set of word vectors that comes with torchtext; it is roughly 2G of data.
# This will take a bit of time
GLOVE = GloVe()
print("Total number of words:",len(GLOVE), "Shape of embeddings:",GLOVE.vectors.shape)

Total number of words: 2196017 Shape of embeddings: torch.Size([2196017, 300])


### Define the model: CNN with GLOVE Embeddings

In [11]:
class CNN1dTextClassificationModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        num_class,
        embed_dim = 300,
        use_pretrained = True,
        fine_tune_embeddings = True,
        debug = False
    ):

        super(CNN1dTextClassificationModel, self).__init__()

        self.embedding = nn.Embedding(
            vocab_size,
            embed_dim,
            padding_idx=PADDING_IDX
        )

        # A linear map from total number of filters (3) to num_class
        self.fc = nn.Linear(3, num_class)

        if use_pretrained:
            # Set the embeddings to not requiring gradients since we'll try and modify
            self.embedding.weight.requires_grad = False
            for i in range(vocab_size):
                # Get the token for the index i
                token = VOCAB.lookup_token(i)
                # Modify the embedding for index i by the embedding for that token
                # Do this only if token is in the stoi dictionary for GLOVE
                if token in GLOVE.stoi:
                   self.embedding.weight[i, :] = GLOVE.get_vecs_by_tokens(TOKENIZER(token),lower_case_backup=True)
            self.embedding.weight.requires_grad = True
        else:
            # Otherwise, initialize the weights as specified below
            self.init_weights()

        # If weights do not get changed, turn off gradients for the GloVe embeddings
        if not fine_tune_embeddings:
          self.embedding.weight.requires_grad = False

        # Define a Conv1d layer that collapses all the channels and does not collapse the time dimension
        self.cnn1 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=1)

        # Define 3 Conv1d layers each having 1 filter and kernel sizes 2, 3 and 4
        self.cnn2 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=2)
        self.cnn3 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=3)
        self.cnn4 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=4)

        self.debug = debug

    def init_weights(self):
        initrange = 0.5
        # Initialize the embedding weight matrix to uniform between the [-0.5, 0.5]
        self.embedding.weight.data.uniform_(-initrange,initrange)

        # Initialize the weight matrix of fc to uniform between the [-0.5, 0.5]
        self.fc.weight.data.uniform_(-initrange,initrange)

        # Initialize the bias for fc to zero
        self.fc.bias.data.fill_(0.0)


    # B = batch_size, L = sequence length, D = vector dimension
    def forward(self, text):
        # B X L X D
        # Get the embeddings for the text passed in
        embedded = self.embedding(text)

        if self.debug:
            print('embedding', embedded.shape)
        # B X D X L
        # Transpose the embedding above as needed
        embedded = embedded.transpose(1, 2)

        # B X 1 X L
        # Pass through cnn1
        cnn1 =  self.cnn1(embedded)
        if self.debug:
            print('cnn1', cnn1.shape)

        # B X 1
        # Take Max pooling over time
        cnn1 = nn.MaxPool1d(kernel_size =cnn1.shape[2], stride=1)(cnn1).squeeze(-1)
        if self.debug:
          print('cnn1 after max pool', cnn1.shape)

        # B X 1 X L - 1
        # Pass through cnn2 and add a RELU
        cnn2 =F.relu(self.cnn2(embedded))
        if self.debug:
            print('cnn2', cnn2.shape)

        # B X 1 X L - 2
        # Pass through cnn3 and add a RELU
        cnn3 = F.relu(self.cnn3(embedded))
        if self.debug:
            print('cnn3', cnn3.shape)

        # B X 1 X L - 3
        # Pass through cnn4 and add a RELU
        cnn4 = F.relu(self.cnn4(embedded))
        if self.debug:
            print('cnn4', cnn4.shape)

        # B X 1 in all cases
        # Apply max pooling to each of cnn2, cnn3 and cnn4
        cnn2 = nn.MaxPool1d(kernel_size =cnn2.shape[2], stride=1)(cnn2).squeeze(-1)
        cnn3 = nn.MaxPool1d(kernel_size =cnn3.shape[2], stride=1)(cnn3).squeeze(-1)
        cnn4 = nn.MaxPool1d(kernel_size =cnn4.shape[2], stride=1)(cnn4).squeeze(-1)

        # B X 1 in all cases
        # Apply max pooling over time
        if self.debug:
            print('cnn2 after max', cnn2.shape)

        # Add to each cnn2, 3, 4 a skip connection to cnn1 and average the results
        cnn2 = (cnn1 + cnn2)/2
        cnn3 = (cnn1 + cnn3)/2
        cnn4 = (cnn1 + cnn4)/2
        if self.debug:
            print('cnn2 after skip connection', cnn2.shape)

        # B X 3
        # Concatenate the above
        cnn_concat = torch.cat((cnn2, cnn3,cnn4), 1)
        if self.debug:
            print('cnn concat', cnn_concat.shape)
            # Set the debug to False after the first forward pass
            self.debug = False

        # Pass through an appropriate Linear layer to get the right dimensions needed
        out = self.fc(cnn_concat)
        return out

### Training the Model

In [12]:
# If this is True, we will initialize the Embedding layer with GLOVE
USE_PRETRANED = True

# If this is True, we will allow for gradient updates on the nn.Embedding layer
FINE_TUNE_EMBEDDINGS = True

# Set the loss appropriately
criterion = nn.CrossEntropyLoss()

model =  CNN1dTextClassificationModel(
        len(VOCAB),
        num_class,
        embed_dim = EMBED_DIM,
        use_pretrained = USE_PRETRANED,
        fine_tune_embeddings = FINE_TUNE_EMBEDDINGS,
        debug = True
    ).to(DEVICE)

# Set the optimizer to SGD
# Add an L2 regularizer of 0.00001
optimizer = torch.optim.SGD(model.parameters(), lr=LR, weight_decay =0.00001)

# Set the scheduler to StepLR with gamma=0.1 and step_size = 1.0
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1.0, gamma=0.1)


def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_acc, total_count = 0, 0
    total_loss, total_batches = 0.0, 0
    total_zero_gradients_percentage = []
    log_interval = 10

    for idx, (label, text) in tqdm(enumerate(dataloader)):
        # Set gradients to zero
        optimizer.zero_grad()

        # Get the predictions
        predicted_label = model(text)

        # Get the loss
        loss = criterion(predicted_label, label)

        # Do back propagation and get the gradients
        loss.backward()

        # Get the loss per batch and the number of batches
        with torch.no_grad():
            total_loss+=loss.item()
            total_batches+=1

        # Loop through all the parameters
        # Specifically, for this batch, get the percentage of zero gradients across all parameters
        # Append this to the list above which will print out the total every 10 batches
        total_nonzero_gradients = 0.0
        total_param_count = 0.0

        for i in model.parameters():
          if i.grad!= None:
            total_nonzero_gradients+= torch.count_nonzero(i.grad).item()
            total_param_count += i.grad.data.numel()

        total_zero_gradients_percentage+=[1-(total_nonzero_gradients/total_param_count)]

        # # Clip the gradient at 10.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), 10.0)

        # Do an optimization step
        optimizer.step()

        # Get the accuracy
        _, prediction = torch.max(predicted_label.data, 1)
        total_acc += (label==prediction).sum().item()

        total_count += label.size(0)

        # Log results
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f} "
                "| loss {:8.3f} "
                "| zero gradients percentage {:8.3f}".format(
                    epoch, idx,
                    len(dataloader),
                    total_acc / total_count,
                    total_loss / total_batches,
                    torch.tensor(total_zero_gradients_percentage).mean().item()
                    )
            )
            # Reset variables as needed
            total_acc, total_count = 0, 0
            total_loss, total_batches = 0.0, 0
            total_zero_gradients = []

### Model Validation:

In [13]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_acc, total_count = 0, 0
    total_loss = 0.0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            # Get the predicted labels
            predicted_label = model(text)
            # Get the accuracy
            _, prediction = torch.max(predicted_label.data, 1)
            loss = criterion(input=predicted_label, target=label)
            total_acc += (label==prediction).sum().item()
            # Update the total count
            total_count +=  label.size(0)
            # Update the total loss
            total_loss += loss
    return total_acc / total_count, total_loss / total_count

### Model Execution:

In [ ]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    accu_val, loss_val = evaluate(valid_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch,
            time.time() - epoch_start_time,
            accu_val,
            loss_val
            )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
accu_test, loss_test = evaluate(test_dataloader, model, criterion)
print("test accuracy {:8.3f}".format(accu_test))
print("test loss {:8.3f}".format(loss_test))

0it [00:00, ?it/s]

embedding torch.Size([128, 75, 300])
cnn1 torch.Size([128, 1, 75])
cnn1 after max pool torch.Size([128, 1])
cnn2 torch.Size([128, 1, 74])
cnn3 torch.Size([128, 1, 73])
cnn4 torch.Size([128, 1, 72])
cnn2 after max torch.Size([128, 1])
cnn2 after skip connection torch.Size([128, 1])
cnn concat torch.Size([128, 3])


11it [00:04,  2.08it/s]

| epoch   1 |    10/  891 batches | accuracy    0.385 | loss    1.266 | zero gradients percentage    0.994


21it [00:10,  1.75it/s]

...

881it [06:54,  1.90it/s]

| epoch   5 |   880/  891 batches | accuracy    0.881 | loss    0.373 | zero gradients percentage    0.992


891it [06:58,  2.13it/s]

| epoch   5 |   890/  891 batches | accuracy    0.909 | loss    0.316 | zero gradients percentage    0.992


-----------------------------------------------------------
| end of epoch   5 | time: 421.52s | valid accuracy    0.876 
-----------------------------------------------------------
Checking the results of test dataset.
test accuracy    0.875
test loss    0.003
